In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shared_utils
from bus_cost_utils import *
from cost_per_bus_nb_scripts import *
from IPython.display import Markdown, display
from matplotlib.ticker import ScalarFormatter
from scipy.stats import zscore
from IPython.display import Markdown, display

In [2]:
merged_data = pd.read_parquet(f'{GCS_PATH}cleaned_no_outliers_cpb_analysis_data_merge.parquet')

In [3]:
# for subsetting ZEB and nonZEB
zeb_list =[
    "BEB",
    "FCEB",
    "electric (not specified)",
    "zero-emission bus (not specified)",
]

non_zeb_list =[
    "CNG",
    "ethanol",
    "low emission (hybrid)",
    "low emission (propane)",
    "mix (zero and low emission)",
]

In [4]:
# means and standard deviations
# for graphs
cpb_mean = merged_data["cost_per_bus"].mean()
cpb_std = merged_data["cost_per_bus"].std()

#testing weighted average calculation for sub-set non-zeb and zeb
zeb_cpb_wt_avg = (merged_data[merged_data["prop_type"].isin(zeb_list)]["total_cost"].sum() / merged_data[merged_data["prop_type"].isin(zeb_list)]["bus_count"].sum())
non_zeb_cpb_wt_avg = (merged_data[merged_data["prop_type"].isin(non_zeb_list)]["total_cost"].sum() / merged_data[merged_data["prop_type"].isin(non_zeb_list)]["bus_count"].sum())

In [5]:
def new_cpb_aggregate(df: pd.DataFrame, column="transit_agency") -> pd.DataFrame:
    """
    function to aggregate compiled data by different categories:
        "transit agency", 
        "propulsion type", 
        "bus_size_type",
        "new_project_type"
    aggregate on columns:
        "project_title"
        "ppno"
        "total_cost"
        "bus_count"
        
    Then, cost per bus is calculated AFTER the aggregation.
    """
    df_agg = (
        df.groupby(column)
        .agg(
            total_project_count=("project_title", "count"),
            total_project_count_ppno=("ppno", "count"),
            total_agg_cost=("total_cost", "sum"),
            total_bus_count=("bus_count", "sum"),
            #new_prop_type=("prop_type","max")
        )
        .reset_index()
    )
    df_agg["new_cost_per_bus"] = (df_agg["total_agg_cost"] / df_agg["total_bus_count"]).astype("int64")
    
    #calculate zscore
    df_agg["new_zscore_cost_per_bus"] = zscore(df_agg["new_cost_per_bus"])
    
    #flag outliers
    df_agg["new_is_cpb_outlier?"] = df_agg["new_zscore_cost_per_bus"].apply(outlier_flag)
    
    return df_agg

In [6]:
def make_chart(y_col: str, title: str, data: pd.DataFrame, x_col: str):
    """
    function to create chart. sorts values by y_col ascending."""
    
    data.sort_values(by=y_col, ascending=False).head(10).plot(
        x=x_col, y=y_col, kind="bar", color="skyblue"
    )
    plt.title(title)
    plt.xlabel(x_col)
    plt.ylabel(y_col)

    plt.ticklabel_format(style="plain", axis="y")
    plt.show()

In [7]:
def dist_curve(
    df: pd.DataFrame,
    mean: str,
    std: str,
    title: str,
    xlabel: str,
):
    """
    function to make distribution curve. uses the "cpb" column of the df.
    """
    sns.histplot(df["cost_per_bus"], kde=True, color="skyblue", bins=20)
    # mean line
    plt.axvline(
        mean, color="red", linestyle="dashed", linewidth=2, label=f"Mean: ${mean:,.2f}"
    )
    # mean+1std
    plt.axvline(
        mean + std,
        color="green",
        linestyle="dashed",
        linewidth=2,
        label=f"Standard Deviation: ${std:,.2f}",
    )
    plt.axvline(mean - std, color="green", linestyle="dashed", linewidth=2)
    plt.axvline(mean + (std * 2), color="green", linestyle="dashed", linewidth=2)
    plt.axvline(mean + (std * 3), color="green", linestyle="dashed", linewidth=2)

    plt.title(title + " with Mean and Standard Deviation")
    plt.xlabel(xlabel)
    plt.ylabel("Frequency")

    # Turn off scientific notation on x-axis?
    plt.gca().xaxis.set_major_formatter(ScalarFormatter(useMathText=False))

    plt.legend()
    plt.show()

    return

In [8]:
# aggregating by big categories
agg_agency = new_cpb_aggregate(merged_data)
agg_prop = new_cpb_aggregate(merged_data, column="prop_type")
agg_bus_size = new_cpb_aggregate(merged_data, column="bus_size_type")
agg_source = new_cpb_aggregate(merged_data, column="source")

In [9]:
# subsetting ZEB and nonZEB data
zeb_projects = merged_data[merged_data["prop_type"].isin(zeb_list)]

non_zeb_projects = merged_data[merged_data["prop_type"].isin(non_zeb_list)]

In [11]:
#pivot table to get totals for each prop type

pivot_prop_type = pd.pivot_table(
    merged_data,
    values = ["bus_count", "total_cost"],
    index = "prop_type",
    aggfunc = "sum",
    margins = True,
    margins_name = "Grand Total"
).reset_index()
pivot_prop_type["cost_per_bus"] = (pivot_prop_type["total_cost"] / pivot_prop_type["bus_count"]).astype("int64")

In [12]:
#pivot for ZEB data

pivot_zeb_prop = pd.pivot_table(
    #filted incoming DF for zeb prop types
    zeb_projects,
    values = ["bus_count", "total_cost"],
    index = "prop_type",
    aggfunc = "sum",
    margins = True,
    margins_name = "Grand Total"
).reset_index()              

pivot_zeb_prop["cost_per_bus"] = (pivot_zeb_prop["total_cost"] / pivot_zeb_prop["bus_count"]).astype("int64")

In [13]:
#pivot for non-ZEB data

pivot_non_zeb_prop = pd.pivot_table(
    #filted incoming DF for non-zeb prop types
    non_zeb_projects,
    values = ["bus_count", "total_cost"],
    index = "prop_type",
    aggfunc = "sum",
    margins = True,
    margins_name = "Grand Total"
).reset_index()

pivot_non_zeb_prop["cost_per_bus"] = (pivot_non_zeb_prop["total_cost"] / pivot_non_zeb_prop["bus_count"]).astype("int64")

In [14]:
# pivot for bus sizes

pivot_size = pd.pivot_table(
    merged_data,
    values = ["bus_count", "total_cost"],
    index = "bus_size_type",
    aggfunc = "sum",
    margins = True,
    margins_name = "Grand Total"
).reset_index()

pivot_size["cost_per_bus"] = (pivot_size["total_cost"] / pivot_size["bus_count"]).astype("int64")

In [15]:
#pivot for data soruces
pivot_source = pd.pivot_table(
    merged_data,
    values = ["bus_count", "total_cost"],
    index = "source",
    aggfunc = "sum",
    margins = True,
    margins_name = "Grand Total"
).reset_index()

pivot_source["cost_per_bus"] = (pivot_source["total_cost"] / pivot_source["bus_count"]).astype("int64")

In [ ]:
# new summary

new_summary = f"""

# Bus Procurement Cost Analysis

## Summary
This analysis examines the cost of buses for transit agencies across the county. Specifically, to observe the variation of bus cost for propulsion type and bus sizes.

Data was compiled from three data sources:
1. FTA Bus and Low- and No-Emission Grant Awards press release (federally funded, nationwide data)
2. TIRCP project data (state-funded, California only)
3. DGS usage report for all procurements from California agencies purchasing from New Flyer and Portera Inc.. 

The initial dataset contained close to 300 projects, but was paired down due to projects including components other than buses. Examples include: projects that constructed new facilities, trainings or other non-bus related items like trains and sea farries were excluded.
The resulting dataset only contained projects that were solely used to procure buses. 

{len(merged_data)} projects were determined to contain solely bus purchases. 
These projects were aggregated against propulsion type and bus size type, and categorized by ZEB and non-ZEB.


Breakdown of each data souce:
{pivot_source.to_markdown(index=False)}


**ZEB buses include:**
- zero-emission (not specified) 
- electric (not specified)
- battery electric 
- fuel cell electric

**Non-ZEB buses include:**
- CNG 
- ethanol 
- ow emission (hybrid, propane) 
- diesel 
- gas

Below are charts and tables that summarize the findings.

"""
display(Markdown(new_summary))

In [16]:
#summary stuff
display(
    Markdown("**ZEB Summary**"),
    pivot_zeb_prop,
    
    Markdown("**Non-ZEB Summary**"),
    pivot_non_zeb_prop,
    
    Markdown("The remaining buses did not specify a propulsion type")
)

**ZEB Summary**

,prop_type,bus_count,total_cost,cost_per_bus
0,BEB,163.0,167232489,1025966
1,FCEB,102.0,120951335,1185797
2,electric (not specified),44.0,56678000,1288136
3,zero-emission bus (not specified),143.0,128156513,896199
4,Grand Total,452.0,473018337,1046500


**Non-ZEB Summary**

,prop_type,bus_count,total_cost,cost_per_bus
0,CNG,252.0,176039140,698568
1,ethanol,9.0,1006750,111861
2,low emission (hybrid),145.0,91824361,633271
3,low emission (propane),44.0,8403969,190999
4,mix (zero and low emission),125.0,36775430,294203
5,Grand Total,575.0,314049650,546173


The remaining buses did not specify a propulsion type

In [17]:
#min max values for all projects
bus_min_max_summary(data=agg_agency,col1="new_cost_per_bus")
bus_min_max_summary(data=agg_agency,col1="total_bus_count")
bus_min_max_summary(data=agg_agency,col1="total_agg_cost")


**Max new_cost_per_bus**

,transit_agency,total_agg_cost,total_bus_count,new_cost_per_bus
71,Transit Joint Powers Authority for Merced County,3223324,2.0,1611662


**Min new_cost_per_bus**

,transit_agency,total_agg_cost,total_bus_count,new_cost_per_bus
45,Oregon Department of Transportation on behalf ...,181250,5.0,36250


**Max total_bus_count**

...

,transit_agency,total_agg_cost,total_bus_count,new_cost_per_bus
9,City of Beloit,653184,1.0,653184
16,City of San Luis Obispo,859270,1.0,859270
49,"SLO TRANSIT (SAN LUIS OBISPO, CA)",847214,1.0,847214


**Max total_agg_cost**

,transit_agency,total_agg_cost,total_bus_count,new_cost_per_bus
24,Dallas Area Rapid Transit (DART),103000000,90.0,1144444


**Min total_agg_cost**

,transit_agency,total_agg_cost,total_bus_count,new_cost_per_bus
45,Oregon Department of Transportation on behalf ...,181250,5.0,36250


In [18]:
## min max values of just ZEB projects
new_cols =[
    "transit_agency",
    "prop_type",
    "total_cost",
    "bus_count",
    "cost_per_bus"]

display(Markdown("**Which Agneices had the highest and lowest cost per bus?**")),
bus_min_max_summary(data=zeb_projects, col1="cost_per_bus", col_list=new_cols)

**Which Agneices had the highest and lowest cost per bus?**

**Max cost_per_bus**

,transit_agency,prop_type,total_cost,bus_count,cost_per_bus
76,University of California - San Diego,BEB,4134000,2.0,2067000


**Min cost_per_bus**

,transit_agency,prop_type,total_cost,bus_count,cost_per_bus
45,City of Wasco,zero-emission bus (not specified),1543000,3.0,514333


In [19]:
display(Markdown(
    "**Which agency procured the most and least amount of ZEBs?**"
))
bus_min_max_summary(data=zeb_projects, col1="bus_count", col_list=new_cols)


**Which agency procured the most and least amount of ZEBs?**

**Max bus_count**

,transit_agency,prop_type,total_cost,bus_count,cost_per_bus
44,City of Los Angeles (LA DOT),zero-emission bus (not specified),102790000,112.0,917767


**Min bus_count**

,transit_agency,prop_type,total_cost,bus_count,cost_per_bus
70,"SLO TRANSIT (SAN LUIS OBISPO, CA)",BEB,847214,1.0,847214
82,City of San Luis Obispo,BEB,859270,1.0,859270


In [20]:
display(Markdown(
    "**Which Agency had the most and least total ZEB cost?**"
))
bus_min_max_summary(data=zeb_projects, col1="total_cost", col_list=new_cols)

**Which Agency had the most and least total ZEB cost?**

**Max total_cost**

,transit_agency,prop_type,total_cost,bus_count,cost_per_bus
44,City of Los Angeles (LA DOT),zero-emission bus (not specified),102790000,112.0,917767


**Min total_cost**

,transit_agency,prop_type,total_cost,bus_count,cost_per_bus
70,"SLO TRANSIT (SAN LUIS OBISPO, CA)",BEB,847214,1.0,847214


In [ ]:
# Charts

In [ ]:
# charts 

In [ ]:
conclusion = f"""
**Conclusion**

Based on these findings, The average cost of a ZEB, throughout the US, is ~$1,000,000, roughly twice the price of a conventional, non-ZEB.
The variance in cost depends mainly on the options the Trasnit
Agencies chooses. Highly optioned/customized buses contribute to high cost.
Unfortunately, analyzing the cost of configuable options is outside the scope of data provided. 
"""
display(
    Markdown(conclusion)
)

-------

In [ ]:
Markdown(summary)

In [ ]:
# all bus distribution
display(Markdown(all_bus_desc))

dist_curve(
    df=no_outliers,
    mean=cpb_mean,
    std=cpb_std,
    title="all buses, cost per bus distribution",
    xlabel="cost per bus, $ million(s)",
)

In [ ]:
# ZEB dist curve
display(Markdown(zeb_desc))

dist_curve(
    df=zeb_no_outliers,
    mean=zeb_only_mean,
    std=zeb_only_std,
    title="ZEB only cost/bus Distribution",
    xlabel="cost per bus, $ million(s)",
)

In [ ]:
# non_zeb distribution
display(Markdown(non_zeb_desc))

dist_curve(
    non_zeb_no_outliers,
    non_zeb_only_mean,
    non_zeb_only_std,
    title="non-ZEB only cost/bus Distribution",
    xlabel='"cost per bus, $ million(s)"',
)

In [ ]:
# COST PER BUS BY PROP TYPE
display(Markdown(cpb_prop_type_desc))
make_chart("cpb", "Cost per bus by propulsion type", x_col="prop_type", data=prop_agg)

In [ ]:
# bus count BY PROP TYPE
display(Markdown(bus_count_prop_type_desc))
make_chart(
    "total_bus_count", 
    "Bus count by propulsion type",
    x_col="prop_type",
    data=prop_agg
)

In [ ]:
display(Markdown(conclusion))